In [ ]:
import pandas as pd

%matplotlib inline

# Pharma ref

In [ ]:
df_pharma = pd.read_csv(
    "../data/documenten_ph/PHARMA_REF.csv",
    usecols=[
        "drug_code", 
        "drug_name_aggregated",
        "nom_produit_pharma", 
        "conditionnement", 
        "nom_firme_pharma", 
        "statut_produit_pharma",
        "code_atc",
        "code_atc_5",
        "code_atc_3",
        "DDD",
    ],
    sep=";",
)

In [ ]:
df_selection = df_pharma[
    (df_pharma.code_atc == "R06AE07")
]
df_selection.DDD = pd.to_numeric(df_selection.DDD.str.extract(r'(\d*),0*')[0])

In [ ]:
df_selection.head()

# Doc ph

In [ ]:
df_doc_ph = pd.read_csv(
    "../data/documenten_ph/DB_V2.TXT",
    usecols=["drug_code", "realization_date", "quantity"],
    encoding="latin_1",
    sep="\t",
)

In [ ]:
df_doc_ph = df_doc_ph[
    (df_doc_ph.drug_code != "#######")
    & (df_doc_ph.realization_date % 10 < 5)
]

df_doc_ph.drug_code = df_doc_ph.drug_code.astype("int64")

df_doc_ph.realization_date = df_doc_ph.realization_date // 10 + df_doc_ph.realization_date % 10 / 4

In [ ]:
df_joined = pd.merge(df_doc_ph, df_selection, on="drug_code")

In [ ]:
df_joined["doses"] = df_joined.quantity * df_joined.DDD

In [ ]:
df_doses = df_joined.groupby(
    ["realization_date", "nom_produit_pharma"]
).agg(
    {"doses": "sum"}
)

df_doses.doses.unstack().plot(marker="o", figsize=(16, 9))